### data preparation - housing data - Grace, Jiebei

### housing data

#### Redfin house sale data, weekly, county and metro level. 

https://www.redfin.com/news/data-center/ 

columns we need: 

region_name,region_type,period_begin,period_end,duration,total_homes_sold,pending_sales,median_sale_price,median_days_to_close. 

 

Data process: 

Check duration is one week. Insert using linear assumption is the duration > 1 week. 

From region_name, split the string to get the state. 

Groupby state(sum county? Or county+metro) and period_end to get aggregated metrics: 

sum(total_homes_sold), sum(pending_sales), mean(median_sale_price), mean(median_days_to_close) 

 

For the mean(), you can use either simply average or weight average where the weight is total_homes_sold.  

 

Smoothing data, add 3 columns for 2, 3 and 4 weeks smoothing. 

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [8]:
df_house = pd.read_csv("weekly_housing_market_data_most_recent.tsv",sep="\t")
print(df_house.shape)

print(df_house["period_begin"].min(), df_house["period_end"].max())

(933295, 78)
2017-01-02 2020-10-18


### covid-19 data 

#### State-Level covid-19 Data from github 

https://github.com/nytimes/covid-19-data 

columns: date,state,fips,cases,deaths 

 

Data process: 

Create dictionary to match the state full name here to the 2 letter abbreviation.  

Add column for the days of week, and column for thee last day of each week (period_end). 

Groupby state and period_end, get the sum(cases), sum(deaths). 

Add cumulative values, using cumsum() 

Smoothing data, add 3 columns for 2, 3 and 4 weeks smoothing. 

 

 

Merge the result from data1 and data2. 

In [9]:
df_covid19 = pd.read_csv("us-states.csv",sep=",")
print(df_covid19.shape)
df_covid19.head()


(13489, 5)


,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0


In [10]:
# merge state_full_abbr
df_name=pd.read_csv("state_full_abbr.csv",sep=",")
df_name=df_name.rename(str.lower, axis='columns')
df_name.head()
df_covid19_merged=df_covid19.merge(df_name, on=["state"])
df_covid19_merged=df_covid19_merged.rename({"state": "state_full_name", "abbreviation": "state"}, axis='columns')
df_covid19_merged['date']=pd.to_datetime(df_covid19_merged['date'])
df_covid19_merged=df_covid19_merged.sort_values(["state","state_full_name", "fips", "date"], ascending=True)
df_covid19_merged.head()

,date,state_full_name,fips,cases,deaths,state
11159,2020-03-12,Alaska,2,1,0,AK
11160,2020-03-13,Alaska,2,1,0,AK
11161,2020-03-14,Alaska,2,1,0,AK
11162,2020-03-15,Alaska,2,1,0,AK
11163,2020-03-16,Alaska,2,3,0,AK


In [11]:
# generate new_cases and new_deaths using diff:
df_covid19_merged["new_cases"]=df_covid19_merged.groupby(["state","state_full_name", "fips"])["cases"].diff(periods=1)
df_covid19_merged["new_deaths"]=df_covid19_merged.groupby(["state","state_full_name", "fips"])["deaths"].diff(periods=1)

df_covid19_merged.head()

,date,state_full_name,fips,cases,deaths,state,new_cases,new_deaths
11159,2020-03-12,Alaska,2,1,0,AK,NaN,NaN
11160,2020-03-13,Alaska,2,1,0,AK,0.0,0.0
11161,2020-03-14,Alaska,2,1,0,AK,0.0,0.0
11162,2020-03-15,Alaska,2,1,0,AK,0.0,0.0
11163,2020-03-16,Alaska,2,3,0,AK,2.0,0.0


In [12]:
# df_covid19_merged_shift=df_covid19_merged.groupby(["state","state_full_name", "fips"]).shift(-1)
# df_covid19_merged_shift=df_covid19_merged_shift.rename({"cases": "cases_shift", "deaths": "deaths_shift"}, axis='columns')


In [13]:
# get the week number and year
print(df_covid19_merged["date"].min(), df_covid19_merged["date"].max())

df_covid19_merged["yyyy_ww"]=pd.DatetimeIndex(df_covid19_merged['date']).strftime('%Y-%U')
# df_covid19_merged['date']=pd.to_datetime(df_covid19_merged['date'])
df_covid19_merged.head()

2020-01-21 00:00:00 2020-11-02 00:00:00


,date,state_full_name,fips,cases,deaths,state,new_cases,new_deaths,yyyy_ww
11159,2020-03-12,Alaska,2,1,0,AK,NaN,NaN,2020-10
11160,2020-03-13,Alaska,2,1,0,AK,0.0,0.0,2020-10
11161,2020-03-14,Alaska,2,1,0,AK,0.0,0.0,2020-10
11162,2020-03-15,Alaska,2,1,0,AK,0.0,0.0,2020-11
11163,2020-03-16,Alaska,2,3,0,AK,2.0,0.0,2020-11


In [14]:
# df_covid19_merged.groupby(["state", "yyyy_ww", "state_full_name", "fips"]).agg({"cases":'sum',"deaths":'sum'})

In [104]:
# group by state and yyyy-ww to get period_end, period_begin, sum(cases), sum(deaths)
df_covid19_state=df_covid19_merged.groupby(["state", "yyyy_ww", "state_full_name", "fips"]).agg(
    covid_period_end=pd.NamedAgg(column="date", aggfunc=max),
    covid_period_begin=pd.NamedAgg(column="date", aggfunc=min),
    new_cases=pd.NamedAgg(column="new_cases", aggfunc=sum),
    new_deaths=pd.NamedAgg(column="new_deaths", aggfunc=sum),
    
    cum_cases=pd.NamedAgg(column="cases", aggfunc=max),
    cum_deaths=pd.NamedAgg(column="deaths", aggfunc=max)).sort_values(["state", "yyyy_ww", "state_full_name", "fips"], ascending=True)

df_covid19_state.head()


covid_period_end covid_period_begin  \
state yyyy_ww state_full_name fips                                       
AK    2020-10 Alaska          2          2020-03-14         2020-03-12   
      2020-11 Alaska          2          2020-03-21         2020-03-15   
      2020-12 Alaska          2          2020-03-28         2020-03-22   
      2020-13 Alaska          2          2020-04-04         2020-03-29   
      2020-14 Alaska          2          2020-04-11         2020-04-05   

                                    new_cases  new_deaths  cum_cases  \
state yyyy_ww state_full_name fips                                     
AK    2020-10 Alaska          2           0.0         0.0          1   
      2020-11 Alaska          2          20.0         0.0         21   
      2020-12 Alaska          2          81.0         1.0        102   
      2020-13 Alaska          2          67.0         2.0        169   
      2020-14 Alaska          2          86.0         3.0        255   

                                    cum_deaths  
state yyyy_ww state_full_name fips              
AK    2020-10 Alaska          2              0  
      2020-11 Alaska          2              0  
      2020-12 Alaska          2              1  
      2020-13 Alaska          2              3  
      2020-14 Alaska          2              6

In [105]:
# smoothing data: 

# df_covid19_state["cum_cases"]=df_covid19_state.groupby(["state", "state_full_name", "fips"])['total_cases'].transform(lambda x: x.cumsum())
# df_covid19_state["cum_deaths"]=df_covid19_state.groupby(["state", "state_full_name", "fips"])['total_deaths'].transform(lambda x: x.cumsum())

df_covid19_state["new_cases_avg_2_wks"]=df_covid19_state.groupby(["state", "state_full_name", "fips"])['new_cases'].transform(lambda x: x.rolling(2).mean())
df_covid19_state["new_cases_avg_3_wks"]=df_covid19_state.groupby(["state", "state_full_name", "fips"])['new_cases'].transform(lambda x: x.rolling(3).mean())
df_covid19_state["new_cases_avg_4_wks"]=df_covid19_state.groupby(["state", "state_full_name", "fips"])['new_cases'].transform(lambda x: x.rolling(4).mean())

df_covid19_state["new_deaths_avg_2_wks"]=df_covid19_state.groupby(["state", "state_full_name", "fips"])['new_deaths'].transform(lambda x: x.rolling(2).mean())
df_covid19_state["new_deaths_avg_3_wks"]=df_covid19_state.groupby(["state", "state_full_name", "fips"])['new_deaths'].transform(lambda x: x.rolling(3).mean())
df_covid19_state["new_deaths_avg_4_wks"]=df_covid19_state.groupby(["state", "state_full_name", "fips"])['new_deaths'].transform(lambda x: x.rolling(4).mean())


df_covid19_state.head()

covid_period_end covid_period_begin  \
state yyyy_ww state_full_name fips                                       
AK    2020-10 Alaska          2          2020-03-14         2020-03-12   
      2020-11 Alaska          2          2020-03-21         2020-03-15   
      2020-12 Alaska          2          2020-03-28         2020-03-22   
      2020-13 Alaska          2          2020-04-04         2020-03-29   
      2020-14 Alaska          2          2020-04-11         2020-04-05   

                                    new_cases  new_deaths  cum_cases  \
state yyyy_ww state_full_name fips                                     
AK    2020-10 Alaska          2           0.0         0.0          1   
      2020-11 Alaska          2          20.0         0.0         21   
      2020-12 Alaska          2          81.0         1.0        102   
      2020-13 Alaska          2          67.0         2.0        169   
      2020-14 Alaska          2          86.0         3.0        255   

                                    cum_deaths  new_cases_avg_2_wks  \
state yyyy_ww state_full_name fips                                    
AK    2020-10 Alaska          2              0                  NaN   
      2020-11 Alaska          2              0                 10.0   
      2020-12 Alaska          2              1                 50.5   
      2020-13 Alaska          2              3                 74.0   
      2020-14 Alaska          2              6                 76.5   

                                    new_cases_avg_3_wks  new_cases_avg_4_wks  \
state yyyy_ww state_full_name fips                                             
AK    2020-10 Alaska          2                     NaN                  NaN   
      2020-11 Alaska          2                     NaN                  NaN   
      2020-12 Alaska          2               33.666667                  NaN   
      2020-13 Alaska          2               56.000000                 42.0   
      2020-14 Alaska          2               78.000000                 63.5   

                                    new_deaths_avg_2_wks  \
state yyyy_ww state_full_name fips                         
AK    2020-10 Alaska          2                      NaN   
      2020-11 Alaska          2                      0.0   
      2020-12 Alaska          2                      0.5   
      2020-13 Alaska          2                      1.5   
      2020-14 Alaska          2                      2.5   

                                    new_deaths_avg_3_wks  new_deaths_avg_4_wks  
state yyyy_ww state_full_name fips                                              
AK    2020-10 Alaska          2                      NaN                   NaN  
      2020-11 Alaska          2                      NaN                   NaN  
      2020-12 Alaska          2                 0.333333                   NaN  
      2020-13 Alaska          2                 1.000000                  0.75  
      2020-14 Alaska          2                 2.000000                  1.50

In [106]:
#convert df to csv, saved to root
df_covid19_state.to_csv('covid19_state.csv')

In [3]:
# read house data
house = pd.read_csv("housing_state.csv")
house['yyyy_ww'] = pd.DatetimeIndex(house['period_begin']).strftime('%Y-%U')

house.head()

,state,period_begin,total_homes_sold,total_value,AVG_median_sales_price,yyyy_ww
0,AK,2017-01-01,81.0,21202000.0,261753.086420,2017-01
1,AK,2017-01-08,95.0,28384895.0,298788.368421,2017-02
2,AK,2017-01-15,86.0,22529900.0,261975.581395,2017-03
3,AK,2017-01-22,105.0,26613000.0,253457.142857,2017-04
4,AK,2017-01-29,116.0,32739500.0,282237.068966,2017-05


In [6]:
house[house['state']=='ND']

,state,period_begin,total_homes_sold,total_value,AVG_median_sales_price,yyyy_ww


In [108]:
# merge covid and house data together:
covid_merge_house = df_covid19_state.merge(house,on=["state","yyyy_ww"],how="left").sort_values(["state", "yyyy_ww"], ascending=True)
covid_merge_house.to_csv("covid_merge_house_2.csv",index=False)
covid_merge_house.head()

,state,yyyy_ww,covid_period_end,covid_period_begin,new_cases,new_deaths,cum_cases,cum_deaths,new_cases_avg_2_wks,new_cases_avg_3_wks,new_cases_avg_4_wks,new_deaths_avg_2_wks,new_deaths_avg_3_wks,new_deaths_avg_4_wks,period_begin,total_homes_sold,total_value,AVG_median_sales_price
0,AK,2020-10,2020-03-14,2020-03-12,0.0,0.0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,2020-03-08,118.0,33587400.0,284638.983051
1,AK,2020-11,2020-03-21,2020-03-15,20.0,0.0,21,0,10.0,NaN,NaN,0.0,NaN,NaN,2020-03-15,130.0,38935000.0,299500.000000
2,AK,2020-12,2020-03-28,2020-03-22,81.0,1.0,102,1,50.5,33.666667,NaN,0.5,0.333333,NaN,2020-03-22,118.0,36995000.0,313516.949153
3,AK,2020-13,2020-04-04,2020-03-29,67.0,2.0,169,3,74.0,56.000000,42.0,1.5,1.000000,0.75,2020-03-29,156.0,47670200.0,305578.205128
4,AK,2020-14,2020-04-11,2020-04-05,86.0,3.0,255,6,76.5,78.000000,63.5,2.5,2.000000,1.50,2020-04-05,112.0,33454050.0,298696.875000


### housing data visualization - trend

#### Housing graph. 
https://www.redfin.com/news/data-center/
Trend plot for sum(total_homes_sold), sum(pending_sales), mean(median_sale_price), mean(median_days_to_close), separately. 

Add column of year for the data. 

For each plot, add the trend for the previous 2 years, so 3 lines in each chart.  

For filers, add data range, state (metro or county), smoothing option (no, 2, 3 or 4 weeks) . 

You will need to use parameter as filter and create calculated filed as row value to have smoothing options works. 


### ARIMA, LSTM, RNN prediction (Pohan)

In [ ]:
# see ARIMA.ipynb (Build the model with training data and evaluate with validation data)
# see LSTM.ipynb (Build the model with training data and evaluate with validation data)
# see final_result_lstm_sarima.ipynb (Build the final models by each state with SARIMA or LSTM)

### Cluster (Yifan)

In [ ]:
# see clustering.ipynb

### Visualization interactive (trend and map)

#### visual
Top chart is the monthly COVID trend. Groupby on US monthly level, with parameter filter to switch the cases, deaths or cumulative cases and deaths. Use this as filter with mouse over selection. The filed is using month. 

The bottom chart is the heat map for use by state. The values are the difference % calculated in prediction model. Different colors are assigned for different clusters. Difference % is used to show shade of color. Filters for cluster number is added. 

In [ ]:
# done in Tableau